In [1]:
# import module

import os
import pickle
import sys
import yara
from tqdm import tqdm

sys.path.append("../GIPS")
from core.feature_extractor import extract_string_lower
from main import GIPS

sys.path.append("../yara")
from gen_yara import genYaraRule

In [2]:
# datasets path

malware_direc_path = '../datasets/PE_malware/'
benign_direc_path = '../datasets/ben/'

In [3]:
# datasets file list

malware_files = os.listdir(malware_direc_path)
malware_files.remove('README.md')
with open('../datasets/pkl/benign_md5.pkl', 'rb') as f:
    benign_files = pickle.load(f)

In [4]:
# split train test

train_files = malware_files[: int(len(malware_files) * 0.8)]
test_files = malware_files[int(len(malware_files) * 0.8): ]
whitelist_files = benign_files[: int(len(benign_files) * 0.8)]
benign_files = benign_files[int(len(benign_files) * 0.8): ]

print(len(train_files), len(test_files), len(whitelist_files), len(benign_files))

23927 5982 23927 5982


In [5]:
# make white list

whitelist = set()

for file in tqdm(whitelist_files):
    path = os.path.join(benign_direc_path, file)
    res = extract_string_lower(path)
    whitelist.update(res)
    
print(len(whitelist))

100%|██████████| 23927/23927 [07:37<00:00, 52.32it/s] 

24033730


In [6]:
# make train data

train = []

for file in tqdm(train_files):
	path = os.path.join(malware_direc_path, file)
	res = extract_string_lower(path)
	
	tmp = []
	for string in res:
		if string not in whitelist:
			tmp.append(string)
			
	train.append(tmp)

100%|██████████| 23927/23927 [03:50<00:00, 103.74it/s]


In [7]:
# parameters

K = 64
M = 2 ** 14
thetaJ = 0.6
window_size = 4
vector_size = 512
eps = 0.4
minpts = 5
ngram = 4
hh1_size = 15000
hh2_size = 15000
hh3_size = 5000
ratio = 0.4

In [8]:
# generate signatures with GIPS

cluster_signatures = GIPS(str_feature=train, window_size=window_size, K=K, M=M, thetaJ=thetaJ, vector_size=vector_size,
    eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio
)

data no: 23927
make minhashed vector


100%|██████████| 23927/23927 [02:37<00:00, 152.19it/s]


checking big group


100%|██████████| 23927/23927 [02:07<00:00, 187.36it/s]


chunking


100%|██████████| 15634/15634 [00:18<00:00, 840.66it/s] 


start DBSCAN
end DBSCAN
make signature


100%|██████████| 18/18 [00:17<00:00,  1.00it/s]


end signature


In [9]:
signatures = set()

for signature_list in cluster_signatures.values():
		for i in signature_list:
			signatures.add(i[0])

print(len(signatures))

13795


In [10]:
# signature text file

sig_list = []
sig_counter = ''

for i in cluster_signatures.values():
    for j in i:
    	sig_list.append(j)
sorted(sig_list, key=lambda x: -x[1])

for i in sig_list:
    sig_counter += i[0] + ', ' + str(i[1]) + '\n'
    
f = open('./sig_counter.txt', 'w')
f.write(sig_counter)
f.close()

In [11]:
# make yara rule

yara_rule = genYaraRule(signatures)

f = open('./demo.yar', 'w')
f.write(yara_rule)
f.close()

In [12]:
rules = yara.compile(filepath='./demo.yar')

In [13]:
# make test data

TP = 0
FN = 0
TP_file = []
yara_file = []
detect_condition = 1

for file in tqdm(test_files):
	
	path = os.path.join(malware_direc_path, file)
	res = extract_string_lower(path)
	
	cnt = 0
	yara_test = ''
	for string in res:
		if string not in whitelist:
			yara_test += string
			if string in signatures:
				cnt += 1
			
	if cnt >= detect_condition:
		TP += 1
		TP_file.append(file)
		rule = rules.match(data=yara_test)
		if rule != []:
			yara_file.append(file)
	else:
		FN += 1

100%|██████████| 5982/5982 [00:58<00:00, 101.43it/s]


In [14]:
print(len(TP_file), len(yara_file))

4694 4657


In [15]:
TP_text = ''

for file in TP_file:
    TP_text += file + '\n'
    
f = open('./TP_list.txt', 'w')
f.write(TP_text)
f.close()

In [16]:
# make benign data

FP = 0
TN = 0
FP_file = []
TN_yara_file = []
detect_condition = 1

for file in tqdm(benign_files):
	
	path = os.path.join(benign_direc_path, file)
	res = extract_string_lower(path)
	
	cnt = 0
	yara_test = ''
	for string in res:
		if string not in whitelist:
			yara_test += string
			if string in signatures:
				cnt += 1
			
	if cnt >= detect_condition:
		FP += 1
		FP_file.append(file)
	else:
		TN += 1
	
	rule = rules.match(data=yara_test)
	if cnt == 0 and rule == []:
		TN_yara_file.append(file)

100%|██████████| 5982/5982 [01:46<00:00, 55.92it/s]


In [17]:
print(len(FP_file), len(TN_yara_file))

69 4855


In [18]:
TN_text = ''

for file in TN_yara_file:
    TN_text += file + '\n'
    
f = open('./TN_list.txt', 'w')
f.write(TN_text)
f.close()

In [19]:
print(f'TP: {TP}\tFN: {FN}')
print(f'FP: {FP}\t\tTN: {TN}')

precision = round(TP / (TP + FP) , 4)
recall = round(TP / (TP + FN), 4)

print(f'precision: {precision}, recall: {recall}')
print('f1-score:', round(2 * (precision * recall) / (precision + recall), 4))

TP: 4694	FN: 1288
FP: 69		TN: 5913
precision: 0.9855, recall: 0.7847
f1-score: 0.8737


In [20]:
clamAV_FN = {'cf5673bec06d5eeb10a8a455a6257640', 'cf92858d59b52667b805ce296d9e0760', 'cfcb10cee93fdfa2fdcd5b3147046ff0', 'd071f6a18f0289b4ee9fc2b047ef59b0', 'd09ade347b9a2ba97128f7559c761940',
 'd112d6e3d1840c23a1f45654b6e38a60', 'd259b25341c678774c7864fe4a2a8b00','d2a569e9e237f4acdf59e372a2138690', 'd3b078da3da457f0b486b15482c02540', 'd40a3bf9abee413fe5f5b0f6648aa200',
 'd531237c96260d0fb5a7faa30bf56680', 'd55d02e4291684db3fe0b9cc451059c0', 'd591b8df86296f2e340c8190c8ea4430', 'd6016565df6958d767f9e01fbe82f8d0', 'd6186f1c3aaf37629ed86ac466571b10',
 'd69638bfc0e4b1fefdc63e8fbfb74a90', 'd70a786451e9030dd18344db00115b70', 'd7377acc36df6bff98eee67bd47aad80', 'd762457c41cc7502fecce7de4797b700', 'd7ea1efcb6ccee4c3f31d4b845980390',
 'd82268ac4f2835c4bcc7fae265996790', 'd890be9659118d799c70bdaabc6b5b30', 'd8c68d23e741f3502d83e50f7378b000', 'd941dc6aa113f9850013e673c609c6b0', 'dc352f9af0b29e5007ee3158dc621eb0',
 'dd1bd5a3b2088ee7fdd25958ddd1b380', 'dd7ce9275a4faa0558152b183446e090', 'dd93c710f5043f5ab8685ab47f699900', 'deac2f54fee7364f32095bf042be4f50', 'df3a294cf08c89d75aee9c65ddc638c0',
 'df5adc62b7e32010936a711fb9ead240', 'e0d59cd368ead491760baabf56d0fd80', 'e11d843e0def883154e9d30ade1101b0', 'e1b1e100055de8bc2ee6030d688b2470', 'e1d782de35060ed43018a1bd30268730',
 'e1f5ba4efa73d737987dcd22a0bc0570', 'e3116c40ff1aa2d781a541b2ed0f6710', 'e36a4451d6c6923bb4f85da854da1b80', 'e39db1f57e8fb943651b12e1bd61eba0', 'e3eade8619e29ce5dda2412b081be810',
 'e3f09f4c17d8a15885503fdbedc7eda0', 'e44c0d3b4710e6da2959807ae99c3f20', 'e4854e87fe7ce900d9bad6ea564a48b0', 'e4959d69754ce9c89654094dec828230', 'e4d3d5c8677fd2d462e48e053fd2f3c0',
 'e4d6bbb7b8b1974a6140e61692ecdf20', 'e5fa5f3a48d11a6cf312d5d51b748e70', 'e61fd9a54e8497db059bb3000ad2d390', 'e63e3fed2ce633e2d2431fe8b2321e50', 'e642d34611da182a342839b320116e30',
 'e6f4adfa8fa76a61b421e9e78d2071f5', 'e73caf3bd02b7766b0ba8c5af1e99e30', 'e74e20062f162470d4aec8be42548b80', 'e82b7f1b14979e2d09149a5835b23bc0', 'e8e6048ea71dd68046976ff24aa9d970',
 'e9209c42907a881db1dc1b74e9e21af0', 'e997ed1153de42dd77f57d088713ced0', 'ea414e647f5caf9ba463d1a808d60f50', 'ea532fc7137c6faa4e5e766df224dbe0', 'ea7e5df4d5a772a9d67ecbc116f03c40',
 'eaf54dd6dfc3b7ddac52839d6cbb9a80', 'eb84f65021ec34fbadf21d6908fccfc0', 'ec4dbb27b223a5f1ee9ec723ec948dd0', 'ed58de4d175dbcbd17f563e0f8218db0', 'edef1ab29538412724e288448ef86190',
 'ee3af545871f002518201d30f0959980', 'eef449ce734eed14d5df002091451540', 'ef63f49e453abaefb4691783b177d790', 'f0b91cb41e8189fbea584dddc132f3a0', 'f0ca239641f6461c9a0f1f1278a989d0',
 'f168ac5e011a37a6a8c26331c0bc61a0', 'f30d04d9438429e221ee292771159a80', 'f390fb2d40fb3894291dd718e4e7b810', 'f3c88d3ed249f9c0721abce841349610', 'f4d5c239805a616eea31c550dc33d270',
 'f52fc4525646c1f3d07993058cb16ef0', 'f5473083645b0494af5ca75ff9304e80', 'f5e3e895ca63f1dfb1e81cf770fd996a', 'f5f34887798f3cc47cde731f4c29a590', 'f5fb498a8e3eae3496ac4936e284e940',
 'f635e9abc88c55658023d2dc83d97b00', 'f639dcff315a92e147da7dfbe1df08d0', 'f65a5fa1bc2de9da2d008dda014d6610', 'f6936c9a81310e8db843826017539dd0', 'f6bba0a5a59139471846578a600db140',
 'f72195da8b5452cfaf53631751213af0', 'f82d2ed9023fe593aaaa1b45527760e0', 'f83bcf49fada09b9b413f1892c4860d0', 'f8917f74c2fff9021337657e65a90610', 'f9485befec2f5102ce474b428241d400',
 'f9a31c1880dabc544882c89ebac6a210', 'fa513ab1c45dfc342e7da252e3079530', 'fa954ba1640c763332b49a4ccfb939c0', 'facc1b29c250b25ff47fbc1986b3af60', 'fb51885e75a06ed962d1988dc300a4c0',
 'fc74b2e404dd8d422c49aabbab7c6470', 'fd1c83285731b6a7a88e50a550f928b0', 'fe92c40f693ab41cb4397706aad30440', 'ff42990903e99e99dd5e7adb6809e8f0', 'ff81ff6b7ef7930be59820348b5236a0'
}

In [21]:
TP_set = set(TP_file)

cnt = 0
for clamAV in clamAV_FN:
    if clamAV not in TP_set:
        print(clamAV)
        cnt += 1
        
print(cnt)

0
